In [ ]:
#!import config/Settings.cs

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta6"

In [ ]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var deploymentName = "practical-task-2-azure-ai";

builder.WithAzureOpenAIChatCompletionService(deploymentName, azureEndpoint, apiKey);

var kernel = builder.Build();

In [ ]:
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planners;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

// Setting Up the Planner
// Load native plugin into the kernel registry, sharing its functions with prompt templates

var planner = new SequentialPlanner(kernel);

In [ ]:
// Using plugins from the repo
using Microsoft.SemanticKernel.AI.ChatCompletion;

var currentDirectory = System.IO.Directory.GetCurrentDirectory();
var pluginsDirectory = Path.Combine(currentDirectory, "Plugins");

var summarizePluginFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "SummarizePlugin");

In [ ]:
var sourceText = File.ReadAllText(Path.Combine(currentDirectory, "Plugins/SummarizePlugin/Summarize/source.txt"));
var result = await kernel.RunAsync(sourceText, summarizePluginFunctions["Summarize"]);
var resultString = result.GetValue<string>();

Console.WriteLine(resultString);